<a href="https://colab.research.google.com/github/hannape/IBAC-Biodiv/blob/master/Kopia_Gridsearch_for_3_representations.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Gridsearch dla każdej reprezentacji osobno. Bo na IBACu wszystkie 3 reprezentacje były puszczone na architekturę z mel-spectrogramu (chyba?)

Na podstawie hania.dldisc: cnn_gridsearch.ipynb , 03 CNN fit and predict (I2)

In [0]:
from google.colab import drive
drive.mount('/content/drive')
from __future__ import print_function

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
from __future__ import print_function

import keras
import numpy
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras.wrappers.scikit_learn import KerasClassifier
from keras import backend as K
from sklearn.model_selection import GridSearchCV

import numpy
import keras
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import Flatten
from keras.layers.convolutional import Conv2D
from keras.layers.convolutional import MaxPooling2D
from keras.utils import np_utils
from keras import backend as K
from sklearn.metrics import roc_curve
from sklearn.metrics import roc_auc_score
from matplotlib import pyplot
#K.set_image_dim_ordering('th')
K.tensorflow_backend.set_image_dim_ordering('th')
import numpy as np
from sklearn import preprocessing
from keras.datasets import mnist
import glob
import os
import random
import pandas as pd
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import tensorflow as tf
from functools import partial, update_wrapper


In [0]:
from numpy.random import seed
seed(667)
from tensorflow import set_random_seed
set_random_seed(667)
import random
random.seed()

'''
#################### Rep 1 - spektro ####################
# rep 1  ------- 63 x 148 ------ TRAIN 17.3k
X_train1 = numpy.load('drive/My Drive/rep IBAC/rep1/X_train_rep1.npy', allow_pickle=True) # 1 and 0 in order
y_train1 = numpy.load('drive/My Drive/rep IBAC/rep1/y_train.npy', allow_pickle=True)
'''
# rep 1V2 ------- 63 x 148 ------ TRAIN 15.9
X_train1 = numpy.load('drive/My Drive/rep IBAC/rep1V2/X_train_rep1.npy', allow_pickle=True) # 1 and 0 in order
y_train1 = numpy.load('drive/My Drive/rep IBAC/rep1V2/y_train_rep1.npy', allow_pickle=True)
'''
# rep 1b ------- 63 x 63 ------
X_train1 = numpy.load('drive/My Drive/rep IBAC/rep1b/X_train_rep1b.npy', allow_pickle=True) # 1 and 0 in order
y_train1 = numpy.load('drive/My Drive/rep IBAC/rep1b/y_train_rep1b.npy', allow_pickle=True)

#################### Rep 3 - mel-spektro ####################

# rep 3 ------- 60 x 111 ------ TRAIN 17.3k
X_train1 = numpy.load('drive/My Drive/rep IBAC/rep3/X_train_rep3.npy', allow_pickle=True) # 1 and 0 in order
y_train1 = numpy.load('drive/My Drive/rep IBAC/rep3/y_train.npy', allow_pickle=True)

# rep 3V2 ------- 60 x 111 ------ TRAIN 15.9
X_train1 = numpy.load('drive/My Drive/rep IBAC/rep3V2/X_train_rep3.npy', allow_pickle=True) # 1 and 0 in order
y_train1 = numpy.load('drive/My Drive/rep IBAC/rep3V2/y_train_rep3.npy', allow_pickle=True)

# rep 3b ------- 60 x 63 ------
X_train1 = numpy.load('drive/My Drive/rep IBAC/rep3b/X_train_rep3b.npy', allow_pickle=True) # 1 and 0 in order
y_train1 = numpy.load('drive/My Drive/rep IBAC/rep3b/y_train_rep3b.npy', allow_pickle=True)

# rep 3V3 ------- 60 x 148 ------
X_train1 = numpy.load('drive/My Drive/rep IBAC/rep3V3/X_train_rep3V3.npy', allow_pickle=True) # 1 and 0 in order
y_train1 = numpy.load('drive/My Drive/rep IBAC/rep3V3/y_train_rep3V3.npy', allow_pickle=True)

#################### Rep 5 - mel-spektro ####################

# rep 5 ------- 64 x 61 ------ TRAIN 17.3k
X_train1 = numpy.load('drive/My Drive/rep IBAC/rep5/X_train_rep5.npy', allow_pickle=True) # 1 and 0 in order
y_train1 = numpy.load('drive/My Drive/rep IBAC/rep5/y_train.npy', allow_pickle=True)

# rep 5V2 ------- 64 x 61 ------ TRAIN 15.9
X_train1 = numpy.load('drive/My Drive/rep IBAC/rep5V2/X_train_rep5.npy', allow_pickle=True) # 1 and 0 in order
y_train1 = numpy.load('drive/My Drive/rep IBAC/rep5V2/y_train_rep5.npy', allow_pickle=True)

# rep 5b ------- 64 x 149 ------
X_train1 = numpy.load('drive/My Drive/rep IBAC/rep5b/X_train_rep5b.npy', allow_pickle=True) # 1 and 0 in order
y_train1 = numpy.load('drive/My Drive/rep IBAC/rep5b/y_train_rep5b.npy', allow_pickle=True)
'''

print('Training set size:')
print(np.shape(X_train1))
print(np.shape(y_train1))


Training set size:
(15904, 63, 148)
(15904,)


In [0]:
r = np.shape(X_train1)[1]
s = np.shape(X_train1)[2]
y_train = y_train1
X_train = X_train1.reshape(X_train1.shape[0], 1, r, s).astype('float32')
print('Training set size:')
print(np.shape(X_train))
print(np.shape(y_train))
input_shape = (1, r, s)
print(input_shape)

Training set size:
(15904, 1, 63, 148)
(15904,)
(1, 63, 148)


In [0]:
# https://github.com/keras-team/keras/issues/2115

### definiowanie wag 
for_zeros = 0.1
for_ones = 0.9
###

### SCORERS

#def binary_crossentropy_weigted(y_true, y_pred, class_weights):
#	y_pred = K.clip(y_pred, K.epsilon(), 1.0 - K.epsilon())
#  
#	loss = K.mean(class_weights*(-y_true * K.log(y_pred) - (1.0 - y_true) * K.log(1.0 - y_pred)),axis=-1)
#	return loss

def weighted_binary_crossentropy( y_true, y_pred, weights_10) :
    y_pred = K.clip(y_pred, K.epsilon(), 1.0 - K.epsilon())
    logloss = -(y_true * K.log(y_pred) * weights_10[0] + (1 - y_true) * K.log(1 - y_pred) * weights_10[1])
    return K.mean(logloss, axis=-1)

#custom_loss1 = partial(binary_crossentropy_weigted, class_weights=np.array([for_zeros,for_ones])) ## scoring for model.compile
#custom_loss1.__name__ ='binary_crossentropy_weigted'

custom_loss2 = partial(weighted_binary_crossentropy, weights_10=np.array([for_ones,for_zeros])) ## scoring for model.compile
custom_loss2.__name__ ='weighted_binary_crossentropy'

## AUC METRIC
def as_keras_metric(method):
    import functools
    from keras import backend as K
    
    @functools.wraps(method)
    def wrapper(self, args, **kwargs):
        """ Wrapper for turning tensorflow metrics into keras metrics """
        value, update_op = method(self, args, **kwargs)
        K.get_session().run(tf.local_variables_initializer())
        with tf.control_dependencies([update_op]):
            value = tf.identity(value)
        return value
    return wrapper
  
auc_roc = as_keras_metric(tf.metrics.auc)

In [0]:
def search_model(dense_layer_sizes, filters, kernel_size, pool_size, hidden_layers, loss_function):
   
    #hidden_layers = 1
    model = Sequential()
    model.add(Conv2D(filters, kernel_size,input_shape=input_shape, activation='relu', data_format='channels_first'))
    model.add(MaxPooling2D(pool_size=pool_size))
    #model.add(Conv2D(filters, kernel_size, activation='relu'))
    #model.add(MaxPooling2D(pool_size=pool_size))
    #model.add(Conv2D(filters, kernel_size, activation='relu'))
    #model.add(MaxPooling2D(pool_size=pool_size))
   
    for i in range(0,hidden_layers):
      # Add one hidden layer
      print('Warstwa '+ str(i+1))
      model.add(Conv2D(filters, kernel_size, activation='relu'))
      model.add(MaxPooling2D(pool_size=pool_size))
  
    model.add(Flatten())
    model.add(Dense(dense_layer_sizes, activation='relu'))
    model.add(Dropout(0.2))
    model.add(Dense(32, activation='relu'))
    model.add(Dropout(0.2))
    model.add(Dense(1, activation='sigmoid'))
    model.compile(loss=loss_function,
                  optimizer='adam',
                  metrics=['accuracy',auc_roc])

    return model

In [0]:
hidden_layers= 2
for i in range(hidden_layers):
  print(i)

0
1


In [0]:
#@title

  
def make_model_modified(dense_layer_sizes, filters, kernel_size, pool_size):
    model = Sequential()
    model.add(Conv2D(filters, kernel_size,input_shape=input_shape, activation='relu', data_format='channels_first')) # "channels_first" corresponds to inputs with shape (batch, channels, height, width)
    model.add(MaxPooling2D(pool_size=pool_size))
    model.add(Conv2D(filters, kernel_size, activation='relu'))
    model.add(MaxPooling2D(pool_size=pool_size))
    model.add(Conv2D(filters, kernel_size, activation='relu'))
    model.add(MaxPooling2D(pool_size=pool_size))
   
    model.add(Flatten())
    model.add(Dense(dense_layer_sizes, activation='relu'))
    model.add(Dropout(0.2))
    model.add(Dense(32, activation='relu'))
    model.add(Dropout(0.2))
    model.add(Dense(1, activation='sigmoid'))
    model.compile(loss='binary_crossentropy',
                  optimizer='adam',
                  metrics=['accuracy'])

    return model
  
def probny_model(dense_layer_sizes):
    model = Sequential()
    model.add(Conv2D(10, (3,3),input_shape=input_shape, activation='relu', data_format='channels_first')) # "channels_first" corresponds to inputs with shape (batch, channels, height, width)
    model.add(MaxPooling2D(pool_size=(2,2)))
   
    model.add(Dense(dense_layer_sizes, activation='relu'))
    model.add(Dropout(0.2))
    model.add(Dense(32, activation='relu'))
    model.add(Dropout(0.2))
    model.add(Dense(1, activation='sigmoid'))
    model.compile(loss='binary_crossentropy',
                  optimizer='adam',
                  metrics=['accuracy'])

    return model  
  
def make_model(dense_layer_sizes, filters, kernel_size, pool_size):

    model = Sequential()
    model.add(Conv2D(filters, kernel_size,
                     padding='valid',
                     input_shape=input_shape, data_format='channels_first'))
    model.add(Activation('relu'))
    model.add(Conv2D(filters, kernel_size))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=pool_size))
    model.add(Dropout(0.25))

    model.add(Flatten())
    for layer_size in dense_layer_sizes:
        model.add(Dense(layer_size))
        model.add(Activation('relu'))
    model.add(Dropout(0.5))
    model.add(Dense(1))#(num_classes))
    model.add(Activation('softmax'))

    model.compile(loss='binary_crossentropy', #'categorical_crossentropy',
                  optimizer='adadelta',
                  metrics=['accuracy'])
    return model  

In [0]:
y=y_train.astype('int')
print(np.shape(X_train))
print(np.shape(y_train))

(15904, 1, 63, 148)
(15904,)


In [0]:
#@title
my_classifier3 = KerasClassifier(make_model)
validator3 = GridSearchCV(my_classifier3,
                         param_grid={'dense_layer_sizes': [128]}, 
                         scoring='neg_log_loss')



In [0]:
#@title
dense_size_candidates = [[32], [64]] #, [32, 32], [64, 64]]
my_classifier = KerasClassifier(make_model, batch_size=32)

validator = GridSearchCV(my_classifier,
                         param_grid={'dense_layer_sizes': [[32], [64]],
                                     # epochs is avail for tuning even when not
                                     # an argument to model building function
                                     'epochs': [1],
                                     'filters': [8],
                                     'kernel_size': [3],
                                     'pool_size': [2]},
                         scoring='neg_log_loss',
                         n_jobs=1)
#y2=y_train.astype('int')
validator.fit(X_train, y) #_binary)

print('The parameters of the best model are: ')
print(validator.best_params_)

In [0]:
from sklearn.model_selection import StratifiedKFold
my_classifier66 = KerasClassifier(search_model)
validator66 = GridSearchCV(my_classifier66,
                         param_grid={'dense_layer_sizes': [128],
                                     # epochs is avail for tuning even when not
                                     # an argument to model building function
                                     'epochs': [15], #[10, 30, 50],
                                     'filters': [10,20], #[10, 20],
                                     'kernel_size': [(3,3)], #[(3,3)],
                                     'pool_size': [(2,2)],#[(2,2)],
                                     'hidden_layers': [2,3],
                                     'loss_function': ['binary_crossentropy', custom_loss2],
                                     'batch_size': [32,64], #[32, 64]
                                     }, 
                         #scoring='neg_log_loss',
                         cv = StratifiedKFold(n_splits = 5, random_state=667, shuffle = True))
#y=y_train.astype('int')
'''
validator66 = GridSearchCV(my_classifier66,
                         param_grid={'dense_layer_sizes': [128, 256],
                                     # epochs is avail for tuning even when not
                                     # an argument to model building function
                                     'epochs': [10,30], #[10, 30, 50],
                                     'filters': [10,20], #[10, 20],
                                     'kernel_size': [(3,3)], #[(3,3)],
                                     'pool_size': [(2,2)],#[(2,2)],
                                     'hidden_layers': [2,3],
                                     'loss_function': ['binary_crossentropy', custom_loss2],
                                     'batch_size': [32,64], #[32, 64]
                                     }, 
                         scoring='neg_log_loss', cv = StratifiedKFold(n_splits =4, random_state=667, shuffle = True))
'''
grid_result = validator66.fit(X_train, y)
print('The parameters of the best model are: ')
print(validator66.best_params_)

means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))


Warstwa 1
Warstwa 2
Epoch 1/15
12723/12723 [==============================] - 11s 897us/step - loss: 0.6419 - acc: 0.6564 - auc: 0.5784
Epoch 2/15
12723/12723 [==============================] - 4s 305us/step - loss: 0.4475 - acc: 0.8033 - auc: 0.7470
Epoch 3/15
12723/12723 [==============================] - 4s 312us/step - loss: 0.3901 - acc: 0.8352 - auc: 0.8034
Epoch 4/15
12723/12723 [==============================] - 4s 316us/step - loss: 0.3576 - acc: 0.8553 - auc: 0.8321
Epoch 5/15
12723/12723 [==============================] - 4s 314us/step - loss: 0.3319 - acc: 0.8683 - auc: 0.8515
Epoch 6/15
12723/12723 [==============================] - 4s 316us/step - loss: 0.3177 - acc: 0.8735 - auc: 0.8658
Epoch 7/15
12723/12723 [==============================] - 4s 322us/step - loss: 0.2993 - acc: 0.8806 - auc: 0.8759
Epoch 8/15
12723/12723 [==============================] - 4s 325us/step - loss: 0.2849 - acc: 0.8885 - auc: 0.8845
Epoch 9/15
12723/12723 [==============================] - 4

In [0]:
print('The parameters of the best model are: ')
print(validator66.best_params_)

means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

The parameters of the best model are: 
{'batch_size': 32, 'dense_layer_sizes': 128, 'epochs': 15, 'filters': 20, 'hidden_layers': 3, 'kernel_size': (3, 3), 'loss_function': 'binary_crossentropy', 'pool_size': (2, 2)}
0.869907 (0.014068) with: {'batch_size': 32, 'dense_layer_sizes': 128, 'epochs': 15, 'filters': 10, 'hidden_layers': 2, 'kernel_size': (3, 3), 'loss_function': 'binary_crossentropy', 'pool_size': (2, 2)}
0.671089 (0.122335) with: {'batch_size': 32, 'dense_layer_sizes': 128, 'epochs': 15, 'filters': 10, 'hidden_layers': 2, 'kernel_size': (3, 3), 'loss_function': functools.partial(<function weighted_binary_crossentropy at 0x7fddec2d7268>, weights_10=array([0.9, 0.1])), 'pool_size': (2, 2)}
0.752012 (0.161275) with: {'batch_size': 32, 'dense_layer_sizes': 128, 'epochs': 15, 'filters': 10, 'hidden_layers': 3, 'kernel_size': (3, 3), 'loss_function': 'binary_crossentropy', 'pool_size': (2, 2)}
0.710827 (0.093366) with: {'batch_size': 32, 'dense_layer_sizes': 128, 'epochs': 15, '

In [0]:
  grid_result.cv_results_['split1_test_score']

In [0]:
print('The parameters of the best model are: ')
print(validator66.best_params_)
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

The parameters of the best model are: 
{'batch_size': 32, 'dense_layer_sizes': 128, 'epochs': 1, 'filters': 10, 'kernel_size': 3, 'pool_size': 2}
-0.696088 (0.005621) with: {'batch_size': 32, 'dense_layer_sizes': 128, 'epochs': 1, 'filters': 10, 'kernel_size': 3, 'pool_size': 2}
-4.680589 (5.632574) with: {'batch_size': 32, 'dense_layer_sizes': 256, 'epochs': 1, 'filters': 10, 'kernel_size': 3, 'pool_size': 2}


In [0]:
# build the model
#del model
#dense_size_candidates = [[32], [64], [32, 32], [64, 64]]
#model = make_model(5, 20, 3, 2)
#model.fit(X_train, y_train, epochs=5, batch_size=32)

my_classifier = KerasClassifier(search_model)
validator = GridSearchCV(my_classifier,
                         param_grid={'dense_layer_sizes': [128, 256],
                                     # epochs is avail for tuning even when not
                                     # an argument to model building function
                                     'epochs': [10, 30, 50],
                                     'filters': [10, 20],
                                     'kernel_size': [(3,3)],
                                     'pool_size': [(2,2)],
                                     'hidden_layers': [1,2,3],
                                     'loss_function': ['binary_crossentropy', custom_loss2],
                                     'batch_size': [[32], [64]]}, 
                         scoring='neg_log_loss')

my_classifier2 = KerasClassifier(make_model_modified)
validator2 = GridSearchCV(my_classifier2,
                         param_grid={'dense_layer_sizes': [128,256],
                                     # epochs is avail for tuning even when not
                                     # an argument to model building function
                                     'epochs': [1,2],
                                     'filters': [20],
                                     'kernel_size': [(3,3)],
                                     'pool_size': [(2,2)],
                                     'batch_size': [32]}, 
                         scoring='neg_log_loss', n_jobs=1)

In [0]:
y_frame = pd.DataFrame(y_train)
y=y_train.astype('int')
grid_result = validator.fit(X_train, y)

In [0]:
'''Example of how to use sklearn wrapper
Builds simple CNN models on MNIST and uses sklearn's GridSearchCV to find best model
'''

from __future__ import print_function

import keras
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras.wrappers.scikit_learn import KerasClassifier
from keras import backend as K
from sklearn.model_selection import GridSearchCV


num_classes = 10

# input image dimensions
img_rows, img_cols = 28, 28

# load training data and do basic data normalization
(x_train, y_train), (x_test, y_test) = mnist.load_data()

if K.image_data_format() == 'channels_first':
    x_train = x_train.reshape(x_train.shape[0], 1, img_rows, img_cols)
    x_test = x_test.reshape(x_test.shape[0], 1, img_rows, img_cols)
    input_shape = (1, img_rows, img_cols)
else:
    x_train = x_train.reshape(x_train.shape[0], img_rows, img_cols, 1)
    x_test = x_test.reshape(x_test.shape[0], img_rows, img_cols, 1)
    input_shape = (img_rows, img_cols, 1)

x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255
print(numpy.shape(x_train))
print(numpy.shape(y_train))

(60000, 28, 28, 1)
(60000,)


In [0]:
a = y_train>1
y_train_binary = a*1
print(y_train_binary)
print(numpy.shape(x_train))
print(numpy.shape(y_train))

[0 0 0 ... 0 0 0]
(60000, 28, 28, 1)
(15904,)


In [0]:

# convert class vectors to binary class matrices
#y_train = keras.utils.to_categorical(y_train, num_classes)
#y_test = keras.utils.to_categorical(y_test, num_classes)


def make_model(dense_layer_sizes, filters, kernel_size, pool_size):
    '''Creates model comprised of 2 convolutional layers followed by dense layers
    dense_layer_sizes: List of layer sizes.
        This list has one number for each layer
    filters: Number of convolutional filters in each convolutional layer
    kernel_size: Convolutional kernel size
    pool_size: Size of pooling area for max pooling
    '''

    model = Sequential()
    model.add(Conv2D(filters, kernel_size,
                     padding='valid',
                     input_shape=input_shape, data_format='channels_first'))
    model.add(Activation('relu'))
    model.add(Conv2D(filters, kernel_size))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=pool_size))
    model.add(Dropout(0.25))

    model.add(Flatten())
    for layer_size in dense_layer_sizes:
        model.add(Dense(layer_size))
        model.add(Activation('relu'))
    model.add(Dropout(0.5))
    model.add(Dense(1))#(num_classes))
    model.add(Activation('softmax'))

    model.compile(loss='binary_crossentropy', #'categorical_crossentropy',
                  optimizer='adadelta',
                  metrics=['accuracy'])
    return model


dense_size_candidates = [[32], [64]] #, [32, 32], [64, 64]]
my_classifier = KerasClassifier(make_model, batch_size=32)
validator = GridSearchCV(my_classifier,
                         param_grid={'dense_layer_sizes': dense_size_candidates,
                                     # epochs is avail for tuning even when not
                                     # an argument to model building function
                                     'epochs': [1],
                                     'filters': [8],
                                     'kernel_size': [3],
                                     'pool_size': [2]},
                         scoring='neg_log_loss',
                         n_jobs=1)
y=y_train.astype('int')
validator.fit(X_train, y) #_binary)

print('The parameters of the best model are: ')
print(validator.best_params_)



In [0]:
# validator.best_estimator_ returns sklearn-wrapped version of best model.
# validator.best_estimator_.model returns the (unwrapped) keras model
best_model = validator.best_estimator_.model
metric_names = best_model.metrics_names
metric_values = best_model.evaluate(x_test, y_test)
for metric, value in zip(metric_names, metric_values):
    print(metric, ': ', value)